## Get the lat long coordinates for each city

In [15]:
import pandas as pd
import os
import gmaps
from pprint import pprint
import requests
import numpy as np

# Google developer API key
from gconfig import gkey

In [16]:
# import csv (merged dataframes).
fi2015 = os.path.join('Resources', 'Census Data','merged_df_2015.csv')
fi2016 = os.path.join('Resources', 'Census Data','merged_df_2016.csv')
fi2017 = os.path.join('Resources', 'Census Data','merged_df_2017.csv')

# read csv file
df_2015 = pd.read_csv(fi2015)
df_2016 = pd.read_csv(fi2016)
df_2017 = pd.read_csv(fi2017)

In [17]:
df_2016.head()

,Unnamed: 0,region,Population,Median Age,Per Capita Income,AveragePrice,Total Volume,year
0,0,Albany,5400.494118,43.880000,31850.588235,1.533942,50618.611442,2016.0
1,1,Atlanta,11380.430868,40.132476,23222.794212,1.214135,272373.829808,2016.0
2,2,BaltimoreWashington,26714.144231,38.093269,44071.394231,1.587596,393209.637692,2016.0
3,3,Boise,27025.300000,37.170000,32924.900000,1.141923,44745.283942,2016.0
4,4,Boston,23225.185185,37.593827,43865.283951,1.426154,293954.952596,2016.0


Replace the name of the city to add state. Some cities had more than one city associated to the field, so a middle city between the two was assigned (see documentation for which city was picked)

In [18]:
replace = {'Albany': 'Albany, New York', 'Atlanta': 'Atlanta, Georgia', 'Boston': 'Boston, Massachusetts', 'HartfordSpringfield': 'Windsor Locks, Connecticut', 'New York': 'New York, New York',
          'Syracuse': 'Syracuse, New York', 'BuffaloRochester': 'Batavia, New York', 'Pittsburgh': 'Pittsburgh, Pennsylvania', 'HarrisburgScranton': 'Hazleton, Pennsylvania', 'Philadelphia': 'Philadelphia, Pennsylvania',
          'BaltimoreWashington': 'Laurel, Maryland', 'SanDiego': 'San Diego, California', 'RichmondNorfolk': 'Jamestown, Virginia', 'Roanoke': 'Roanoke, Virginia', 'RaleighGreensboro': 'Hillsborough, North Carolina',
          'Charlotte': 'Charlotte, North Carolina','SouthCarolina': 'Columbia, South Carolina', 'Atlanta': 'Atlanta, Georgia', 'Jacksonville': 'Jacksonville, Florida', 'Orlando': 'Orlando, Florida',
          'MiamiFtLauderdale': 'Miami, Florida', 'Tampa': 'Tampa, Florida', 'Nashville': 'Nashville, Tennessee', 'Louisville': 'Louisville, Kentucky', 'CincinnatiDayton': 'Monroe, Ohio',
          'Columbus': 'Columbus, Ohio', 'Indianapolis': 'Indianapolis, Indiana', 'Detroit': 'Detroit, Michigan', 'Grand Rapids': 'Grand Rapids, Michigan', 'Chicago': 'Chicago, Illinois',
          'GreatLakes': 'Milwaukee, Wisconsin', 'StLouis': 'St Louis, Missouri', 'NewOrleansMobile': 'New Orleans, Louisiana', 'DallasFtWorth': 'Dallas, Texas', 'Houston': 'Houston, Texas',
          'Denver': 'Denver, Colorado', 'Boise': 'Boise, Idaho', 'PhoenixTucson': 'Eloy, Arizona', 'WestTexNewMexico': 'Albuquerque, New Mexico', 'LasVegas': 'Las Vegas, Nevada',
          'Los Angeles': 'Los Angeles, California', 'SanFrancisco': 'San Francisco, California', 'Sacramento': 'Sacramento, California', 'California': 'Fresno, California',
          'Portland': 'Portland, Oregon', 'Seattle': 'Seattle, Washington', 'Spokane': 'Spokane, Washington'}

df_2015 = df_2015.replace({'region' : replace})
df_2016 = df_2016.replace({'region' : replace})
df_2017 = df_2017.replace({'region' : replace})

In [19]:
# citites with state stored
target_2015 = df_2015['region'].to_list()
# target_2016 = df_2016['region'].to_list()
# target_2017 = df_2017['region'].to_list()

In [21]:
# empty list to append lat long values
lat = []
lng = []
city = target_2015
#url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(address, gkey)
# parameters to be used in the Geocode API
 
key = gkey

# Execute request using the URL and dictionary containing query parameters
for address in city:
    response = requests.get(('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(address, gkey)).json()
    lat.append(response["results"][0]["geometry"]["location"]["lat"])
    lng.append(response["results"][0]["geometry"]["location"]["lng"])


In [22]:
# Store the lat long with city in a dict
lat_long = {"region": target_2015,
           "latitude": lat,
           "longitude": lng}

In [23]:
# Make a data frame from dict
df = pd.DataFrame(lat_long)

In [26]:
# Merge the two dataframes on region to add the lat lon information
summary_df_2015 = df_2015.merge(df, on='region')
summary_df_2016 = df_2016.merge(df, on='region')
summary_df_2017 = df_2017.merge(df, on='region')
summary_df_2015.head()

,Unnamed: 0,region,Population,Median Age,Per Capita Income,AveragePrice,Total Volume,year,latitude,longitude
0,0,"Albany, New York",5355.093023,43.820930,31027.627907,1.538750,38749.004135,2015.0,42.652579,-73.756232
1,1,"Atlanta, Georgia",11295.530547,39.880064,22475.807074,1.380577,223381.712692,2015.0,33.748995,-84.387982
2,2,"Laurel, Maryland",26444.240385,38.003846,43212.625000,1.368846,390822.880192,2015.0,39.099275,-76.848306
3,3,"Boise, Idaho",26614.300000,37.280000,31414.700000,1.373750,36388.051346,2015.0,43.615019,-116.202314
4,4,"Boston, Massachusetts",23028.012346,37.492593,42649.234568,1.473558,263990.304231,2015.0,42.360082,-71.058880


### Heat Map of Total Volume

In [27]:
gmaps.configure(gkey)

In [28]:
# Store 'Lat' and 'Lng' into a variable 
locations_2015 = summary_df_2015[["latitude", "longitude"]]
locations_2016 = summary_df_2016[["latitude", "longitude"]]
locations_2017 = summary_df_2017[["latitude", "longitude"]]


# Store median age value in a variable 
age_2015 = summary_df_2015["Median Age"]
age_2016 = summary_df_2016["Median Age"]
age_2017 = summary_df_2017["Median Age"]

# Store Total Volume in variable
total_2015 = summary_df_2015["Total Volume"]
total_2016 = summary_df_2016["Total Volume"]
total_2017 = summary_df_2017["Total Volume"]

# Store Income in variable
income_2015 = summary_df_2015["Per Capita Income"]
income_2016 = summary_df_2016["Per Capita Income"]
income_2017 = summary_df_2017["Per Capita Income"]

2015 Heat Maps (Total Volume, Income Per Capita)

In [29]:
# Create the mapping figure
fig = gmaps.figure()

# Create the heatmap layer using locations and total volume
heat_layer_2015 = gmaps.heatmap_layer(locations=locations_2015, weights=total_2015, 
                                 dissipating=False, max_intensity=200000,
                                 point_radius=0.9)

fig.add_layer(heat_layer_2015)

fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
fig = gmaps.figure()
g = [(80, 80 ,245, 0),
     (255, 30 ,30, 1),
    (6, 6, 116, 1),
    (1, 1, 19, 1)]


# Create the heatmap layer using locations and income per capita
heat_layer2_2015 = gmaps.heatmap_layer(locations=locations_2015, weights=income_2015,  
                                 dissipating=False, max_intensity=53000,
                                 point_radius=1, gradient = g)


fig.add_layer(heat_layer2_2015)

fig

Figure(layout=FigureLayout(height='420px'))

2016 Heat Maps (Total Volume, Income Per Capita)

In [31]:
# Create the mapping figure
fig = gmaps.figure()

# Create the heatmap layer using locations and total volume
heat_layer_2016 = gmaps.heatmap_layer(locations=locations_2016, weights=total_2016, 
                                 dissipating=False, max_intensity=200000,
                                 point_radius=0.9)

fig.add_layer(heat_layer_2016)

fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
fig = gmaps.figure()
g = [(80, 80 ,245, 0),
     (255, 30 ,30, 1),
    (6, 6, 116, 1),
    (1, 1, 19, 1)]


# Create the heatmap layer using locations and income per capita
heat_layer2_2016 = gmaps.heatmap_layer(locations=locations_2016, weights=income_2016,  
                                 dissipating=False, max_intensity=53000,
                                 point_radius=1, gradient = g)


fig.add_layer(heat_layer2_2016)

fig

Figure(layout=FigureLayout(height='420px'))

2017 Heat Maps (Total Volume, Income Per Capita)

In [33]:
# Create the mapping figure
fig = gmaps.figure()

# Create the heatmap layer using locations and total volume
heat_layer_2017 = gmaps.heatmap_layer(locations=locations_2017, weights=total_2017, 
                                 dissipating=False, max_intensity=200000,
                                 point_radius=0.9)

fig.add_layer(heat_layer_2017)

fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
fig = gmaps.figure()
g = [(80, 80 ,245, 0),
     (255, 30 ,30, 1),
    (6, 6, 116, 1),
    (1, 1, 19, 1)]


# Create the heatmap layer using locations and income per capita
heat_layer2_2017 = gmaps.heatmap_layer(locations=locations_2017, weights=income_2017,  
                                 dissipating=False, max_intensity=53000,
                                 point_radius=1, gradient = g)


fig.add_layer(heat_layer2_2017)

fig

Figure(layout=FigureLayout(height='420px'))

## Joint maps of Total consumption (total volume) and Income Per Capita

In [35]:
fig = gmaps.figure()

fig.add_layer(heat_layer_2015)
fig.add_layer(heat_layer2_2015)

fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
fig = gmaps.figure()

fig.add_layer(heat_layer_2016)
fig.add_layer(heat_layer2_2016)

fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
fig = gmaps.figure()

fig.add_layer(heat_layer_2017)
fig.add_layer(heat_layer2_2017)

fig

Figure(layout=FigureLayout(height='420px'))